In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 11:36:06.568189


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:
- Regression deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

Variables:
1. olefin feed (barrels per day), denoted as $x\_1$
2. isobutane recycle (barrels per day), denoted as $x\_2$
3. acid addition rate (thousands of pounds per day), denoted as $x\_3$
4. alkylate yield (barrels per day), denoted as $x\_4$
5. isobutane makeup (barrels per day), denoted as $x\_5$
6. acid strength (weight per cent), denoted as $x\_6$
7. motor octane number, denoted as $x\_7$
8. external isobutane-to-olefin ratio, denoted as $x\_8$
9. acid dilution factor, denoted as $x\_9$
10. F-4 performance number, denoted as $x\_{10}$

These parameters and variables will be used to formulate the optimization model. Next, we'll define the objective function.

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:
- Regression deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

Variables:
1. olefin feed (barrels per day), denoted as $x\_1$
2. isobutane recycle (barrels per day), denoted as $x\_2$
3. acid addition rate (thousands of pounds per day), denoted as $x\_3$
4. alkylate yield (barrels per day), denoted as $x\_4$
5. isobutane makeup (barrels per day), denoted as $x\_5$
6. acid strength (weight per cent), denoted as $x\_6$
7. motor octane number, denoted as $x\_7$
8. external isobutane-to-olefin ratio, denoted as $x\_8$
9. acid dilution factor, denoted as $x\_9$
10. F-4 performance number, denoted as $x\_{10}$

These parameters and variables will be used to formulate the optimization model. Next, we'll define the objective function.


In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective function is defined as the profit, which is calculated as the revenue generated from the alkylate yield multiplied by the motor octane number, minus the operational costs. The revenue can be expressed as the product of alkylate yield ($x\_4$) and motor octane number ($x\_7$). The operational costs include the costs of olefin feed ($x\_1$), isobutane recycle ($x\_2$), acid addition rate ($x\_3$), and isobutane makeup ($x\_5$).

Considering the deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, the objective function can be expressed as:

$$\text{maximize} \quad (x\_4 \cdot (1 + 0.02Y\_1)) \cdot (x\_7 \cdot (1 + 0.01Y\_2)) - (c\_1 \cdot x\_1 + c\_2 \cdot x\_2 + c\_3 \cdot x\_3 + c\_5 \cdot x\_5)$$

where $c\_1$, $c\_2$, $c\_3$, and $c\_5$ are the respective unit costs of olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, and $Y\_1$ and $Y\_2$ are random variables representing the deviation from the true values of alkylate yield and motor octane number, respectively.

The objective function is to maximize the profit under the uncertainty of the deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

We will now move on to step 3, defining the constraints.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective function is defined as the profit, which is calculated as the revenue generated from the alkylate yield multiplied by the motor octane number, minus the operational costs. The revenue can be expressed as the product of alkylate yield ($x\_4$) and motor octane number ($x\_7$). The operational costs include the costs of olefin feed ($x\_1$), isobutane recycle ($x\_2$), acid addition rate ($x\_3$), and isobutane makeup ($x\_5$).

Considering the deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, the objective function can be expressed as:

$$\text{maximize} \quad (x\_4 \cdot (1 + 0.02Y\_1)) \cdot (x\_7 \cdot (1 + 0.01Y\_2)) - (c\_1 \cdot x\_1 + c\_2 \cdot x\_2 + c\_3 \cdot x\_3 + c\_5 \cdot x\_5)$$

where $c\_1$, $c\_2$, $c\_3$, and $c\_5$ are the respective unit costs of olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, and $Y\_1$ and $Y\_2$ are rando

In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

The constraints for this optimization model are derived from the relationships between variables and the bounds on each variable:

1. Alkylate yield ($x\_4$) is a function of olefin feed ($x\_1$) and external isobutane-to-olefin ratio ($x\_8$). The relationship is defined as:

$$x\_4 = x\_1 \cdot (1.12 + 0.13167x\_8 - 0.00667x\_8^2)$$

2. Motor octane number ($x\_7$) is derived from external isobutane-to-olefin ratio ($x\_8$) and acid strength ($x\_6$). The relationship is defined as:

$$x\_7 = 86.35 + 1.098x\_8 - 0.038x\_8^2 + 0.325(x\_6 - 89)$$

3. Acid dilution factor ($x\_9$) is calculated based on the F-4 performance number ($x\_{10}$). The relationship is defined as:

$$x\_9 = 35.82 - 0.222x\_{10}$$

4. F-4 performance number ($x\_{10}$) depends on the motor octane number ($x\_7$). The relationship is defined as:

$$x\_{10} = -133 + 3x\_7$$

5. The external isobutane-to-olefin ratio ($x\_8$) needs to equal the ratio of isobutane recycle ($x\_2$) plus isobutane makeup ($x\_5$) to olefin feed ($x\_1$):

$$x\_8 = \frac{x\_2 + x\_5}{x\_1}$$

6. The acid strength ($x\_6$) needs to equal the ratio of 93000 times acid addition rate ($x\_3$) to acid addition rate ($x\_3$) multiplied by acid dilution factor ($x\_9$) in addition to 1000 times acid addition rate ($x\_3$):

$$x\_6 = \frac{93000x\_3}{x\_3 \cdot x\_9 + 1000x\_3}$$

7. Lastly, 1.22 alkylate yield ($x\_4$) needs to be equal to the combined olefin feed ($x\_1$) and isobutane makeup ($x\_5$):

$$1.22x\_4 = x\_1 + x\_5$$

8. Each variable has a lower and an upper bound as per the nature of the chemical process:

$$x\_{i, min} \leq x\_i \leq x\_{i, max} \quad (i = 1, 2, \ldots, 10)$$

These constraints ensure that the optimization model accurately reflects the relationships between variables and the limitations imposed by the chemical process.

The objective function and constraints together form the complete optimization model.

In [12]:
print(response_3_text)

Step 3: Define the constraints

The constraints for this optimization model are derived from the relationships between variables and the bounds on each variable:

1. Alkylate yield ($x\_4$) is a function of olefin feed ($x\_1$) and external isobutane-to-olefin ratio ($x\_8$). The relationship is defined as:

$$x\_4 = x\_1 \cdot (1.12 + 0.13167x\_8 - 0.00667x\_8^2)$$

2. Motor octane number ($x\_7$) is derived from external isobutane-to-olefin ratio ($x\_8$) and acid strength ($x\_6$). The relationship is defined as:

$$x\_7 = 86.35 + 1.098x\_8 - 0.038x\_8^2 + 0.325(x\_6 - 89)$$

3. Acid dilution factor ($x\_9$) is calculated based on the F-4 performance number ($x\_{10}$). The relationship is defined as:

$$x\_9 = 35.82 - 0.222x\_{10}$$

4. F-4 performance number ($x\_{10}$) depends on the motor octane number ($x\_7$). The relationship is defined as:

$$x\_{10} = -133 + 3x\_7$$

5. The external isobutane-to-olefin ratio ($x\_8$) needs to equal the ratio of isobutane recycle ($x\_2$) pl

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:
- Regression deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

Variables:
1. olefin feed (barrels per day), denoted as $x\_1$
2. isobutane recycle (barrels per day), denoted as $x\_2$
3. acid addition rate (thousands of pounds per day), denoted as $x\_3$
4. alkylate yield (barrels per day), denoted as $x\_4$
5. isobutane makeup (barrels per day), denoted as $x\_5$
6. acid strength (weight per cent), denoted as $x\_6$
7. motor octane number, denoted as $x\_7$
8. external isobutane-to-olefin ratio, denoted as $x\_8$
9. acid dilution factor, denoted as $x\_9$
10. F-4 performance number, denoted as $x\_{10}$

These parameters and variables will be used to formulate the optimization model. Next, we'll define the objective function.
Step 2: Define the objective function

The objective function is defined as the profit, which is calculated as the revenue generated from the alkylate yield multiplied by the motor octane number, minus the operational costs. The revenue can be expressed as the product of alkylate yield ($x\_4$) and motor octane number ($x\_7$). The operational costs include the costs of olefin feed ($x\_1$), isobutane recycle ($x\_2$), acid addition rate ($x\_3$), and isobutane makeup ($x\_5$).

Considering the deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, the objective function can be expressed as:

$$\text{maximize} \quad (x\_4 \cdot (1 + 0.02Y\_1)) \cdot (x\_7 \cdot (1 + 0.01Y\_2)) - (c\_1 \cdot x\_1 + c\_2 \cdot x\_2 + c\_3 \cdot x\_3 + c\_5 \cdot x\_5)$$

where $c\_1$, $c\_2$, $c\_3$, and $c\_5$ are the respective unit costs of olefin feed, isobutane recycle, acid addition rate, and isobutane makeup, and $Y\_1$ and $Y\_2$ are random variables representing the deviation from the true values of alkylate yield and motor octane number, respectively.

The objective function is to maximize the profit under the uncertainty of the deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

We will now move on to step 3, defining the constraints.
Step 3: Define the constraints

The constraints for this optimization model are derived from the relationships between variables and the bounds on each variable:

1. Alkylate yield ($x\_4$) is a function of olefin feed ($x\_1$) and external isobutane-to-olefin ratio ($x\_8$). The relationship is defined as:

$$x\_4 = x\_1 \cdot (1.12 + 0.13167x\_8 - 0.00667x\_8^2)$$

2. Motor octane number ($x\_7$) is derived from external isobutane-to-olefin ratio ($x\_8$) and acid strength ($x\_6$). The relationship is defined as:

$$x\_7 = 86.35 + 1.098x\_8 - 0.038x\_8^2 + 0.325(x\_6 - 89)$$

3. Acid dilution factor ($x\_9$) is calculated based on the F-4 performance number ($x\_{10}$). The relationship is defined as:

$$x\_9 = 35.82 - 0.222x\_{10}$$

4. F-4 performance number ($x\_{10}$) depends on the motor octane number ($x\_7$). The relationship is defined as:

$$x\_{10} = -133 + 3x\_7$$

5. The external isobutane-to-olefin ratio ($x\_8$) needs to equal the ratio of isobutane recycle ($x\_2$) plus isobutane makeup ($x\_5$) to olefin feed ($x\_1$):

$$x\_8 = \frac{x\_2 + x\_5}{x\_1}$$

6. The acid strength ($x\_6$) needs to equal the ratio of 93000 times acid addition rate ($x\_3$) to acid addition rate ($x\_3$) multiplied by acid dilution factor ($x\_9$) in addition to 1000 times acid addition rate ($x\_3$):

$$x\_6 = \frac{93000x\_3}{x\_3 \cdot x\_9 + 1000x\_3}$$

7. Lastly, 1.22 alkylate yield ($x\_4$) needs to be equal to the combined olefin feed ($x\_1$) and isobutane makeup ($x\_5$):

$$1.22x\_4 = x\_1 + x\_5$$

8. Each variable has a lower and an upper bound as per the nature of the chemical process:

$$x\_{i, min} \leq x\_i \leq x\_{i, max} \quad (i = 1, 2, \ldots, 10)$$

These constraints ensure that the optimization model accurately reflects the relationships between variables and the limitations imposed by the chemical process.

The objective function and constraints together form the complete optimization model.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:
- Regression deviation percentages for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number.

Variables:
1. olefin feed (barrels per day), denoted as $x\_1$
2. isobutane recycle (barrels per day), denoted as $x\_2$
3. acid addition rate (thousands of pounds per day), denoted as $x\_3$
4. alkylate yield (barrels per day), denoted as $x\_4$
5. isobutane makeup (barrels per day), denoted as $x\_5$
6. acid strength (weight per cent), denoted as $x\_6$
7. motor octane number, denoted as $x\_7$
8. external isobutane-to-olefin ratio, denoted as $x\_8$
9. acid dilution factor, denoted as $x\_9$
10. F-4 performance number, denoted as $x\_{10}$

These parameters and variables will be used to formulate the optimization model. Next, we'll define the objective function.
Step 2: Define the objective function

The objective function is defined as the profit, which is calculated as the revenue generated from the alky

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Step 1: Define parameters and variables
model = pyo.ConcreteModel()

# Define parameters (using sample data)
model.Y1 = 1.0  # Regression deviation percentage for alkylate yield
model.Y2 = 1.0  # Regression deviation percentage for motor octane number

# Define unit costs (using sample data)
model.c1 = 1.0  # Unit cost of olefin feed
model.c2 = 1.0  # Unit cost of isobutane recycle
model.c3 = 1.0  # Unit cost of acid addition rate
model.c5 = 1.0  # Unit cost of isobutane makeup

# Define variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Olefin feed
model.x2 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Isobutane recycle
model.x3 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid addition rate
model.x4 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Alkylate yield
model.x5 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Isobutane makeup
model.x6 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid strength
model.x7 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Motor octane number
model.x8 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # External isobutane-to-olefin ratio
model.x9 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid dilution factor
model.x10 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # F-4 performance number

# Step 2: Define the objective function
model.obj = pyo.Objective(expr=(model.x4 * (1 + 0.02 * model.Y1)) * (model.x7 * (1 + 0.01 * model.Y2)) - (model.c1 * model.x1 + model.c2 * model.x2 + model.c3 * model.x3 + model.c5 * model.x5), sense=pyo.maximize)

# Step 3: Define the constraints
model.c1 = pyo.Constraint(expr=model.x4 == model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8 ** 2))
model.c2 = pyo.Constraint(expr=model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8 ** 2 + 0.325 * (model.x6 - 89))
model.c3 = pyo.Constraint(expr=model.x9 == 35.82 - 0.222 * model.x10)
model.c4 = pyo.Constraint(expr=model.x10 == -133 + 3 * model.x7)
model.c5 = pyo.Constraint(expr=model.x8 == (model.x2 + model.x5) / model.x1)
model.c6 = pyo.Constraint(expr=model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3))
model.c7 = pyo.Constraint(expr=1.22 * model.x4 == model.x1 + model.x5)

# Step 4: Solve the optimization problem
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Step 5: Print the results
print("Optimal Solution:")
print("Profit: ", pyo.value(model.obj))
print("Olefin feed: ", pyo.value(model.x1))
print("Isobutane recycle: ", pyo.value(model.x2))
print("Acid addition rate: ", pyo.value(model.x3))
print("Alkylate yield: ", pyo.value(model.x4))
print("Isobutane makeup: ", pyo.value(model.x5))
print("Acid strength: ", pyo.value(model.x6))
print("Motor octane number: ", pyo.value(model.x7))
print("External isobutane-to-olefin ratio: ", pyo.value(model.x8))
print("Acid dilution factor: ", pyo.value(model.x9))
print("F-4 performance number: ", pyo.value(model.x10))
```

This code defines the optimization problem using Pyomo and solves it using the GLPK solver. The results of the optimization are printed out after solving the problem.

In [18]:
print(response_4_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Step 1: Define parameters and variables
model = pyo.ConcreteModel()

# Define parameters (using sample data)
model.Y1 = 1.0  # Regression deviation percentage for alkylate yield
model.Y2 = 1.0  # Regression deviation percentage for motor octane number

# Define unit costs (using sample data)
model.c1 = 1.0  # Unit cost of olefin feed
model.c2 = 1.0  # Unit cost of isobutane recycle
model.c3 = 1.0  # Unit cost of acid addition rate
model.c5 = 1.0  # Unit cost of isobutane makeup

# Define variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Olefin feed
model.x2 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Isobutane recycle
model.x3 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid addition rate
model.x4 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Alkylate yield
model.x5 = pyo.Var(within=pyo.NonNegativeReals

### Code Executability

In [20]:
import pyomo.environ as pyo

# Step 1: Define parameters and variables
model = pyo.ConcreteModel()

# Define parameters (using sample data)
model.Y1 = 1.0  # Regression deviation percentage for alkylate yield
model.Y2 = 1.0  # Regression deviation percentage for motor octane number

# Define unit costs (using sample data)
model.c1 = 1.0  # Unit cost of olefin feed
model.c2 = 1.0  # Unit cost of isobutane recycle
model.c3 = 1.0  # Unit cost of acid addition rate
model.c5 = 1.0  # Unit cost of isobutane makeup

# Define variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Olefin feed
model.x2 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Isobutane recycle
model.x3 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid addition rate
model.x4 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Alkylate yield
model.x5 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Isobutane makeup
model.x6 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid strength
model.x7 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Motor octane number
model.x8 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # External isobutane-to-olefin ratio
model.x9 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # Acid dilution factor
model.x10 = pyo.Var(within=pyo.NonNegativeReals, bounds=(0, None))  # F-4 performance number

# Step 2: Define the objective function
model.obj = pyo.Objective(expr=(model.x4 * (1 + 0.02 * model.Y1)) * (model.x7 * (1 + 0.01 * model.Y2)) - (model.c1 * model.x1 + model.c2 * model.x2 + model.c3 * model.x3 + model.c5 * model.x5), sense=pyo.maximize)

# Step 3: Define the constraints
model.c1 = pyo.Constraint(expr=model.x4 == model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8 ** 2))
model.c2 = pyo.Constraint(expr=model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8 ** 2 + 0.325 * (model.x6 - 89))
model.c3 = pyo.Constraint(expr=model.x9 == 35.82 - 0.222 * model.x10)
model.c4 = pyo.Constraint(expr=model.x10 == -133 + 3 * model.x7)
model.c5 = pyo.Constraint(expr=model.x8 == (model.x2 + model.x5) / model.x1)
model.c6 = pyo.Constraint(expr=model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3))
model.c7 = pyo.Constraint(expr=1.22 * model.x4 == model.x1 + model.x5)

# Step 4: Solve the optimization problem
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Step 5: Print the results
print("Optimal Solution:")
print("Profit: ", pyo.value(model.obj))
print("Olefin feed: ", pyo.value(model.x1))
print("Isobutane recycle: ", pyo.value(model.x2))
print("Acid addition rate: ", pyo.value(model.x3))
print("Alkylate yield: ", pyo.value(model.x4))
print("Isobutane makeup: ", pyo.value(model.x5))
print("Acid strength: ", pyo.value(model.x6))
print("Motor octane number: ", pyo.value(model.x7))
print("External isobutane-to-olefin ratio: ", pyo.value(model.x8))
print("Acid dilution factor: ", pyo.value(model.x9))
print("F-4 performance number: ", pyo.value(model.x10))

on block (model).unknown with a new Component
with type <class 'pyomo.core.base.constraint.AbstractScalarConstraint'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del unknown.c1
on block (model).unknown with a new Component
with type <class 'pyomo.core.base.constraint.AbstractScalarConstraint'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del unknown.c2
on block (model).unknown with a new Component
with type <class 'pyomo.core.base.constraint.AbstractScalarConstraint'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del unknown.c3
on block (model).unknown with a new Component
with type <class 'pyomo.core.base.constraint.AbstractScalarConstraint'>.
This is usually indicative of a modelling error.
To avoid this warning, explicitly delete the attribute:
    del unknown.c5
  - termination condition: m

Optimal Solution:
Profit:  316087845678910.3
Olefin feed:  2997091866558.774
Isobutane recycle:  84607982945.27779
Acid addition rate:  0.04953241899492686
Alkylate yield:  3537060008662.186
Isobutane makeup:  1318121344009.0923
Acid strength:  92.37464688439091
Motor octane number:  87.95232976771786
External isobutane-to-olefin ratio:  0.4680267709538517
Acid dilution factor:  6.769748374699905
F-4 performance number:  130.85698930315357


### Solution Correctness